# 视频1

In [1]:
import keras

Using TensorFlow backend.


In [2]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [3]:
 datagen = ImageDataGenerator(
         rotation_range=0.2,
         width_shift_range=0.2,
         height_shift_range=0.2,
         shear_range=0.2,
         zoom_range=0.2,
         horizontal_flip=True,
         fill_mode='nearest')
#ImageDataGenerator　实现了大多数上文中提到的图像几何变换方法．
#rotation_range: 旋转范围, 随机旋转(0-180)度;
#width_shift and height_shift: 随机沿着水平或者垂直方向，以图像的长宽小部分百分比为变化范围进行平移;
#rescale: 对图像按照指定的尺度因子, 进行放大或缩小, 设置值在0 - 1之间，通常为1 / 255;
#shear_range: 水平或垂直投影变换
#zoom_range: 按比例随机缩放图像尺寸;
#horizontal_flip: 水平翻转图像;
#fill_mode: 填充像素, 出现在旋转或平移之后

In [4]:
img = load_img(r'./cat.jpg')  
# 这里的路径是我放置的示例图片，请选择你自己的图片路径

In [5]:
x = img_to_array(img) 
print(x.shape)
#将图片转化为矩阵进行处理

(360, 480, 3)


In [6]:
x = x.reshape((1,) + x.shape)
print(x.shape)

(1, 360, 480, 3)


In [7]:
i = 0
for batch in datagen.flow(x,
                           batch_size=1,
                           save_to_dir=r'./result/',
                           #生成后的图像保存路径
                           save_prefix='cat',
                           save_format='jpg'):
    i += 1
    if i > 20:
        break  

# 视频2

In [8]:
import warnings

from keras.layers import Conv2D
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Input
from keras.layers import MaxPooling2D
from keras.models import Model


# 实验二：用 keras 搭建一个简单的 CNN 网络
def simple_cnn(input_shape, classes, weights_path=None):

    # 输入层。shape 为函数参数值。
    img_input = Input(shape=input_shape, name='cifar10_input')

    # Block 1
    # 两个 3*3 kernel_size 的卷积层，步长为 1*1，矩阵维度不变，filter 数量为 32
    #    后跟一个最大池化层，kernel_size 为 2*2，步长为 2*2
    x = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', name='block1_conv1')(img_input)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', name='block1_conv2')(x)
    # TODO:

    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    # 两个 3*3 kernel_size 的卷积层，步长为 1*1，矩阵维度不变，filter 数量为 64
    #    后跟一个最大池化层，kernel_size 为 2*2，步长为 2*2
    #    之后加一个 dropout，dropout_rate 设为 0.25
    # TODO:
    x = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', name='block2_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    x = Dropout(0.25)(x)

    # Block 3
    # 两个 3*3 kernel_size 的卷积层，步长为 1*1，矩阵维度不变，filter 数量为 128
    #    后跟一个最大池化层，kernel_size 为 2*2，步长为 2*2
    #    之后加一个 dropout，dropout_rate 设为 0.25
    # TODO:
    x = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', name='block3_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', name='block3_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    x = Dropout(0.25)(x)
    # FC Block
    # 全连接层，第一层的神经元的数量为 1024，加一个 dropout，dropout_rate 为 0.5
    #    然后是输出的全连接层，神经元数量为函数参数 classes，分类器使用 softmax
    x = Flatten(name='flatten')(x)

    # TODO:
    x = Dense(1024, activation='relu', kernel_initializer='he_normal', name='fc')(x)
    x = Dropout(0.5)(x)
    x = Dense(classes, activation='softmax', kernel_initializer='he_normal', name='prediction_cifar10')(x)

    model = Model(img_input, x, name='simple_cnn')
    if weights_path:
        model.load_weights(weights_path, by_name=False)
    return model


# VERY DEEP CONVOLUTIONAL NETWORKS FOR LARGE-SCALE IMAGE RECOGNITION
def VGG16(input_shape=(224, 224, 3), classes=1000, weights_path=None, dropout=0.5):

    img_input = Input(shape=input_shape, name='cifar10_input')

    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    x = Flatten(name='flatten')(x)
    x = Dense(4096, activation='relu', kernel_initializer='he_normal', name='new_fc1')(x)
    x = Dropout(dropout)(x)
    x = Dense(4096, activation='relu', kernel_initializer='he_normal', name='new_fc2')(x)
    x = Dropout(dropout)(x)
    x = Dense(
        classes, activation='softmax', kernel_initializer='he_normal', name='prediction_cifar10')(x)

    model = Model(img_input, x, name='vgg16')
    if not weights_path:
        warnings.warn('The weights_path param is invalid.')
    else:
        model.load_weights(weights_path, by_name=True)
    return model

In [9]:
cnn = simple_cnn((32,32,3), 10)
cnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cifar10_input (InputLayer)   (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 32)        896       
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 32)        9248      
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 32)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 64)        18496     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 64)        36928     
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 64)          0         
__________

# 视频3

In [10]:
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator

from keras.datasets import cifar10

VGG_WEIGHTS_PATH = './vgg16.h5'
CNN_WEIGHTS_PATH = './simple_cnn.h5'
NUM_CLASSES = 10
IMG_SHAPE = (32, 32, 3)

In [11]:
if __name__ == '__main__':
    ((x_train, y_train), (x_valid, y_valid)) = cifar10.load_data()
    y_train = keras.utils.to_categorical(y_train, NUM_CLASSES)
    y_valid = keras.utils.to_categorical(y_valid, NUM_CLASSES)

    print('simple shape:', x_train.shape[1:])
    print('train samples: {}; valid samples: {}'.format(
        x_train.shape[0], x_valid.shape[0]))

    model=simple_cnn(IMG_SHAPE, NUM_CLASSES)
    #learning rate
    optimizer=optimizers.rmsprop(decay=1e-6,lr=1e-4)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.summary()

('simple shape:', (32, 32, 3))
train samples: 50000; valid samples: 10000
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cifar10_input (InputLayer)   (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 32)        896       
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 32)        9248      
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 32)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 64)        18496     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 64)        36928     
_________________________________________________________________
bl

In [12]:
print (y_train.shape)
if __name__ == '__main__':
    model.fit(
        x_train,
        y_train,
        #batch_size
        batch_size = 256,
        #epochs
        epochs = 2,
        validation_data=(x_valid, y_valid),
        shuffle=True)
    
    model.save(model.name + '.h5')
        
    #train_VGG16(
        #model=models.VGG16(IMG_SHAPE, NUM_CLASSES, VGG_WEIGHTS_PATH, dropout=1.0),
        #batch_size=256,
        #epochs=13,
        #optimizer=optimizers.Adam(lr=1e-4, decay=1e-6),
    #)

(50000, 10)
Train on 50000 samples, validate on 10000 samples
Epoch 1/2
50000/50000 [==============================] - 9s 179us/step - loss: 14.5080 - acc: 0.0999 - val_loss: 14.5063 - val_acc: 0.1000
Epoch 2/2
50000/50000 [==============================] - 6s 123us/step - loss: 14.5063 - acc: 0.1000 - val_loss: 14.5063 - val_acc: 0.1000


# 视频4

In [13]:
from PIL import Image
import numpy as np
from keras.models import load_model

MODEL_PATH='simple_cnn.h5'
IMG_PATH = 'cat2.jpg'

d= {0:'airplane',
   1:'automobile',
   2:'bird',
   3:'cat',
   4:'deer',
   5:'dog',
   6:'frog',
   7:'horse',
   8:'ship',
   9:'truck'}

In [14]:
model =load_model(MODEL_PATH)
model.summary()
img=Image.open(IMG_PATH)
img

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cifar10_input (InputLayer)   (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 32)        896       
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 32)        9248      
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 32)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 64)        18496     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 64)        36928     
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 64)          0         
__________

<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=768x768 at 0x7EFBC87FB7A0>

In [15]:
img = img.resize((32, 32))  
data=np.array(img)
data=data.reshape(1,32,32,3)

In [16]:
result=model.predict(data).round(0)
result=np.where(result[0]==1)
result=result[0].tolist()[0]
print (('>>> the predict result is : {}').format(d[result]))

>>> the predict result is : airplane


In [17]:
from keras.models import load_model
from keras.datasets import cifar10

MODEL_PATH='simple_cnn.h5'

((x_train, y_train), (x_test, y_test)) = cifar10.load_data()
y_train = keras.utils.to_categorical(y_train, NUM_CLASSES)
y_test = keras.utils.to_categorical(y_test, NUM_CLASSES)

In [18]:
model =load_model('simple_cnn.h5')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cifar10_input (InputLayer)   (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 32)        896       
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 32)        9248      
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 32)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 64)        18496     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 64)        36928     
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 64)          0         
__________

In [19]:
scores = model.evaluate(x_test, y_test, verbose=1)
print(('valid loss:{}, valid accuracy:{}').format(scores[0], scores[1]))

10000/10000 [==============================] - 1s 114us/step
valid loss:14.5062856644, valid accuracy:0.1
